# 1. IMPORT AND INSTALL DEPENDENCIES 

In [1]:
!pip install tensorflow-gpu 

  Using cached tensorflow_gpu-2.9.1-cp310-cp310-win_amd64.whl (444.1 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\dell\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [2]:
!pip install opencv-python 

In [3]:
!pip  install --user mediapipe

In [4]:
!pip install matplotlib

In [5]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. KEYPOINTS USING MP HOLISTIC

In [6]:
mp_holistic=mp.solutions.holistic   #Holistic Model
mp_drawing=mp.solutions.drawing_utils  #Drawing utilities

In [7]:
def mediapipe_detection(image,model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)#color conversion BGR to RGB
    image.flags.writeable=False#Image is no longer writeable
    results=model.process(image)#Make prediction
    image.flags.writeable=True#Image is no longer writeable
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)#Color conversion RGN to BGR
    return image,results

In [8]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)#Draw face connections
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)#Draw pose connections
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)#Draw left hands connections
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)#Draw right hands connections

In [9]:
def draw_styled_landmarks(image,results):
    #Draw face connections
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                            mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                            mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1))
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius=4),
                            mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),
                            mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                            mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))    

In [ ]:
cap=cv2.VideoCapture(0)
#Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #Read feed
        ret,frame=cap.read()
        #Make detections 
        image,results=mediapipe_detection(frame,holistic)
        print(results)
        #Draw Landmarks
        draw_styled_landmarks(image, results)
        #show on screen
        cv2.imshow('OpenCV Feed',image)
        #Break 
        if cv2.waitKey(10)&0xFF==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [11]:
draw_landmarks(frame,results)

In [ ]:
plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))

# 3. EXTRACT KEYPOINT VALUES

In [13]:
len(results.face_landmarks.landmark)

468

In [14]:
len(results.pose_landmarks.landmark)

33

In [15]:
pose=[]
for res in results.pose_landmarks.landmark:
    test=np.array([res.x,res.y,res.z,res.visibility])
    pose.append(test)

In [16]:
pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)


In [17]:
def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [18]:
result_test=extract_keypoints(results)

In [19]:
result_test

array([ 0.48559803,  0.70681792, -1.23216927, ...,  0.        ,
        0.        ,  0.        ])

In [20]:
np.save('0', result_test)

In [21]:
np.load('0.npy')

array([ 0.48559803,  0.70681792, -1.23216927, ...,  0.        ,
        0.        ,  0.        ])

# 4.SETUP FOLDERS FOR COLLECTION

In [22]:
#Path for exported data,numpy arrays
DATA_PATH=os.path.join('MP_Data')
#Actions that we try to detect
actions=np.array(['hello','thanks','iloveyou'])
#Third videos worth of data
no_sequences=30
#Videos are going to be 30 frames in length
sequence_length=30

In [23]:
#hello
#0
#1
#2
#...
#29
#thanks
#I love you

In [24]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:pass

# 5. COLLECT KEYPOINT VALUES FOR TRAINING AND TESTING

In [25]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [26]:
cap.release()
cv2.destroyAllWindows()

# 6. PREPROCESS DATA AND CREATE LABELS AND FEATURES

In [27]:
!pip install sklearn

In [28]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [29]:
label_map={label:num for num,label in enumerate(actions)}


In [30]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [31]:
sequences,labels=[],[]
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res=np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])    

In [32]:
np.array(sequences).shape

(90, 30, 1662)

In [33]:
np.array(labels).shape

(90,)

In [34]:
X=np.array(sequences)

In [35]:
X.shape

(90, 30, 1662)

In [36]:
y=to_categorical(labels).astype(int)

In [37]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [38]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.05)

In [39]:
y_test.shape

(5, 3)

# 7. BUILD AND TRAIN LSTM NEURAL NETWORK

In [40]:
!pip install tensorboard

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [42]:
log_dir=os.path.join('Logs1')
tb_callback=TensorBoard(log_dir=log_dir)

In [43]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [44]:
res=[.7,0.2,0.1]

In [45]:
actions[np.argmax(res)]

'hello'

In [46]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [47]:
model.fit(X_train,y_train,epochs=2000,callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 10s 134ms/step - loss: 2.7154 - categorical_accuracy: 0.3412
Epoch 2/2000
3/3 [==============================] - 0s 132ms/step - loss: 12.8293 - categorical_accuracy: 0.3412
Epoch 3/2000
3/3 [==============================] - 0s 120ms/step - loss: 1.8326 - categorical_accuracy: 0.3529
Epoch 4/2000
3/3 [==============================] - 0s 110ms/step - loss: 1.9256 - categorical_accuracy: 0.2471
Epoch 5/2000
3/3 [==============================] - 0s 115ms/step - loss: 1.3351 - categorical_accuracy: 0.4118
Epoch 6/2000
3/3 [==============================] - 0s 127ms/step - loss: 1.8063 - categorical_accuracy: 0.4118
Epoch 7/2000
3/3 [==============================] - 0s 117ms/step - loss: 1.8995 - categorical_accuracy: 0.3176
Epoch 8/2000
3/3 [==============================] - 0s 121ms/step - loss: 1.2209 - categorical_accuracy: 0.4118
Epoch 9/2000
3/3 [==============================] - 0s 122ms/step - loss: 0.9730 - categorical_accurac

3/3 [==============================] - 0s 128ms/step - loss: 0.8982 - categorical_accuracy: 0.5765
Epoch 74/2000
3/3 [==============================] - 0s 123ms/step - loss: 0.8530 - categorical_accuracy: 0.6471
Epoch 75/2000
3/3 [==============================] - 0s 125ms/step - loss: 0.8142 - categorical_accuracy: 0.6235
Epoch 76/2000
3/3 [==============================] - 0s 116ms/step - loss: 0.7189 - categorical_accuracy: 0.7176
Epoch 77/2000
3/3 [==============================] - 0s 131ms/step - loss: 0.6951 - categorical_accuracy: 0.7059
Epoch 78/2000
3/3 [==============================] - 0s 127ms/step - loss: 0.7292 - categorical_accuracy: 0.7294
Epoch 79/2000
3/3 [==============================] - 0s 129ms/step - loss: 0.8273 - categorical_accuracy: 0.6824
Epoch 80/2000
3/3 [==============================] - 0s 130ms/step - loss: 0.7803 - categorical_accuracy: 0.7294
Epoch 81/2000
3/3 [==============================] - 0s 134ms/step - loss: 0.8020 - categorical_accuracy: 0.67

3/3 [==============================] - 0s 130ms/step - loss: 1.0128 - categorical_accuracy: 0.5529
Epoch 146/2000
3/3 [==============================] - 0s 118ms/step - loss: 1.2412 - categorical_accuracy: 0.4588
Epoch 147/2000
3/3 [==============================] - 0s 119ms/step - loss: 1.3755 - categorical_accuracy: 0.4824
Epoch 148/2000
3/3 [==============================] - 0s 134ms/step - loss: 1.1342 - categorical_accuracy: 0.3765
Epoch 149/2000
3/3 [==============================] - 0s 114ms/step - loss: 1.1309 - categorical_accuracy: 0.3412
Epoch 150/2000
3/3 [==============================] - 0s 135ms/step - loss: 1.0694 - categorical_accuracy: 0.4824
Epoch 151/2000
3/3 [==============================] - 0s 124ms/step - loss: 1.0345 - categorical_accuracy: 0.3529
Epoch 152/2000
3/3 [==============================] - 0s 124ms/step - loss: 1.0177 - categorical_accuracy: 0.4588
Epoch 153/2000
3/3 [==============================] - 0s 122ms/step - loss: 1.0007 - categorical_accura

3/3 [==============================] - 0s 124ms/step - loss: 0.5656 - categorical_accuracy: 0.8000
Epoch 218/2000
3/3 [==============================] - 0s 127ms/step - loss: 0.6188 - categorical_accuracy: 0.7059
Epoch 219/2000
3/3 [==============================] - 0s 114ms/step - loss: 30.6328 - categorical_accuracy: 0.6824
Epoch 220/2000
3/3 [==============================] - 0s 135ms/step - loss: 1.1953 - categorical_accuracy: 0.4824
Epoch 221/2000
3/3 [==============================] - 0s 127ms/step - loss: 1.7060 - categorical_accuracy: 0.3059
Epoch 222/2000
3/3 [==============================] - 0s 124ms/step - loss: 1.3685 - categorical_accuracy: 0.5059
Epoch 223/2000
3/3 [==============================] - 0s 125ms/step - loss: 1.6360 - categorical_accuracy: 0.4000
Epoch 224/2000
3/3 [==============================] - 0s 120ms/step - loss: 1.2757 - categorical_accuracy: 0.4235
Epoch 225/2000
3/3 [==============================] - 0s 114ms/step - loss: 1.2284 - categorical_accur

Epoch 289/2000
3/3 [==============================] - 0s 128ms/step - loss: 0.9250 - categorical_accuracy: 0.5765
Epoch 290/2000
3/3 [==============================] - 0s 138ms/step - loss: 0.9281 - categorical_accuracy: 0.5294
Epoch 291/2000
3/3 [==============================] - 0s 134ms/step - loss: 0.8676 - categorical_accuracy: 0.5882
Epoch 292/2000
3/3 [==============================] - 0s 120ms/step - loss: 0.8888 - categorical_accuracy: 0.5765
Epoch 293/2000
3/3 [==============================] - 0s 124ms/step - loss: 0.8978 - categorical_accuracy: 0.5647
Epoch 294/2000
3/3 [==============================] - 0s 130ms/step - loss: 0.8680 - categorical_accuracy: 0.5647
Epoch 295/2000
3/3 [==============================] - 0s 120ms/step - loss: 0.8630 - categorical_accuracy: 0.6353
Epoch 296/2000
3/3 [==============================] - 0s 135ms/step - loss: 0.8234 - categorical_accuracy: 0.6588
Epoch 297/2000
3/3 [==============================] - 0s 116ms/step - loss: 0.8119 - cat

...

3/3 [==============================] - 0s 104ms/step - loss: 9.5757e-06 - categorical_accuracy: 1.0000
Epoch 1682/2000
3/3 [==============================] - 0s 91ms/step - loss: 9.5533e-06 - categorical_accuracy: 1.0000
Epoch 1683/2000
3/3 [==============================] - 0s 91ms/step - loss: 9.5575e-06 - categorical_accuracy: 1.0000
Epoch 1684/2000
3/3 [==============================] - 0s 97ms/step - loss: 9.5252e-06 - categorical_accuracy: 1.0000
Epoch 1685/2000
3/3 [==============================] - 0s 102ms/step - loss: 9.4986e-06 - categorical_accuracy: 1.0000
Epoch 1686/2000
3/3 [==============================] - 0s 113ms/step - loss: 9.4874e-06 - categorical_accuracy: 1.0000
Epoch 1687/2000
3/3 [==============================] - 0s 88ms/step - loss: 9.4649e-06 - categorical_accuracy: 1.0000
Epoch 1688/2000
3/3 [==============================] - 0s 94ms/step - loss: 9.4425e-06 - categorical_accuracy: 1.0000
Epoch 1689/2000
3/3 [==============================] - 0s 93ms/step -

3/3 [==============================] - 0s 124ms/step - loss: 8.4174e-06 - categorical_accuracy: 1.0000
Epoch 1751/2000
3/3 [==============================] - 0s 94ms/step - loss: 8.3963e-06 - categorical_accuracy: 1.0000
Epoch 1752/2000
3/3 [==============================] - 0s 123ms/step - loss: 8.3851e-06 - categorical_accuracy: 1.0000
Epoch 1753/2000
3/3 [==============================] - 0s 96ms/step - loss: 8.3683e-06 - categorical_accuracy: 1.0000
Epoch 1754/2000
3/3 [==============================] - 0s 99ms/step - loss: 8.3528e-06 - categorical_accuracy: 1.0000
Epoch 1755/2000
3/3 [==============================] - 0s 95ms/step - loss: 8.3360e-06 - categorical_accuracy: 1.0000
Epoch 1756/2000
3/3 [==============================] - 0s 95ms/step - loss: 8.3178e-06 - categorical_accuracy: 1.0000
Epoch 1757/2000
3/3 [==============================] - 0s 97ms/step - loss: 8.3024e-06 - categorical_accuracy: 1.0000
Epoch 1758/2000
3/3 [==============================] - 0s 97ms/step - 

3/3 [==============================] - 0s 92ms/step - loss: 7.4020e-06 - categorical_accuracy: 1.0000
Epoch 1820/2000
3/3 [==============================] - 0s 102ms/step - loss: 7.3796e-06 - categorical_accuracy: 1.0000
Epoch 1821/2000
3/3 [==============================] - 0s 95ms/step - loss: 7.3712e-06 - categorical_accuracy: 1.0000
Epoch 1822/2000
3/3 [==============================] - 0s 108ms/step - loss: 7.3557e-06 - categorical_accuracy: 1.0000
Epoch 1823/2000
3/3 [==============================] - 0s 97ms/step - loss: 7.3361e-06 - categorical_accuracy: 1.0000
Epoch 1824/2000
3/3 [==============================] - 0s 96ms/step - loss: 7.3249e-06 - categorical_accuracy: 1.0000
Epoch 1825/2000
3/3 [==============================] - 0s 102ms/step - loss: 7.3165e-06 - categorical_accuracy: 1.0000
Epoch 1826/2000
3/3 [==============================] - 0s 100ms/step - loss: 7.3039e-06 - categorical_accuracy: 1.0000
Epoch 1827/2000
3/3 [==============================] - 0s 87ms/step 

3/3 [==============================] - 0s 97ms/step - loss: 6.4989e-06 - categorical_accuracy: 1.0000
Epoch 1889/2000
3/3 [==============================] - 0s 93ms/step - loss: 6.4834e-06 - categorical_accuracy: 1.0000
Epoch 1890/2000
3/3 [==============================] - 0s 89ms/step - loss: 6.4736e-06 - categorical_accuracy: 1.0000
Epoch 1891/2000
3/3 [==============================] - 0s 104ms/step - loss: 6.4610e-06 - categorical_accuracy: 1.0000
Epoch 1892/2000
3/3 [==============================] - 0s 96ms/step - loss: 6.4540e-06 - categorical_accuracy: 1.0000
Epoch 1893/2000
3/3 [==============================] - 0s 110ms/step - loss: 6.4400e-06 - categorical_accuracy: 1.0000
Epoch 1894/2000
3/3 [==============================] - 0s 89ms/step - loss: 6.4288e-06 - categorical_accuracy: 1.0000
Epoch 1895/2000
3/3 [==============================] - 0s 95ms/step - loss: 6.4161e-06 - categorical_accuracy: 1.0000
Epoch 1896/2000
3/3 [==============================] - 0s 101ms/step -

3/3 [==============================] - 0s 106ms/step - loss: 5.7219e-06 - categorical_accuracy: 1.0000
Epoch 1958/2000
3/3 [==============================] - 0s 95ms/step - loss: 5.7065e-06 - categorical_accuracy: 1.0000
Epoch 1959/2000
3/3 [==============================] - 0s 90ms/step - loss: 5.6925e-06 - categorical_accuracy: 1.0000
Epoch 1960/2000
3/3 [==============================] - 0s 106ms/step - loss: 5.6827e-06 - categorical_accuracy: 1.0000
Epoch 1961/2000
3/3 [==============================] - 0s 93ms/step - loss: 5.6743e-06 - categorical_accuracy: 1.0000
Epoch 1962/2000
3/3 [==============================] - 0s 98ms/step - loss: 5.6630e-06 - categorical_accuracy: 1.0000
Epoch 1963/2000
3/3 [==============================] - 0s 116ms/step - loss: 5.6560e-06 - categorical_accuracy: 1.0000
Epoch 1964/2000
3/3 [==============================] - 0s 109ms/step - loss: 5.6448e-06 - categorical_accuracy: 1.0000
Epoch 1965/2000
3/3 [==============================] - 0s 89ms/step 

In [48]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

In [49]:
res=model.predict(X_train)

3/3 [==============================] - 1s 33ms/step


In [50]:
actions[np.argmax(res[0])]

'hello'

# 9. SAVING WEIGHTS

In [51]:
model.save('action.h5')

In [52]:
model.load_weights('action.h5')

# 10. EVALUATION USING CONFUSION MATRIX AND ACCURACY 

In [53]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

In [54]:
yhat=model.predict(X_test)

1/1 [==============================] - 0s 55ms/step


In [55]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [56]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 1]],

       [[2, 0],
        [0, 3]],

       [[4, 0],
        [0, 1]]], dtype=int64)

In [57]:
accuracy_score(ytrue, yhat)

1.0

# 11. TEST IN REALTIME

In [58]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res,actions,image,colors))

In [60]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 30ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 33ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
thanks
<class 'media

1/1 [==============================] - 0s 24ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
thanks
<class 'media

1/1 [==============================] - 0s 28ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 32ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
thanks
<class 'media

1/1 [==============================] - 0s 29ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
iloveyou
<cla

1/1 [==============================] - 0s 24ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
thanks
<class 'm

1/1 [==============================] - 0s 22ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 36ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 36ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
thanks
